In [1]:
import pandas as pd
from datetime import datetime

# Load Excel
df = pd.read_excel("data/emission_factors.xlsx")

In [2]:
df.head()

,Section,Material,Unit of Material,Year/Timeline,Q1 Quantity,Emission Factor,Unit of Emission Factor,GHG Emission (tCO2),Carbon Content (%),Combustion Efficiency (%),Data Source for Emission Factor,Location (Plant)
0,Pellet Plant,Anthracite Coal,tonnes,Q1,136843.66,1.044,tCO2/t,1.428648e+05,88.13,89.56,IPCC 2006 Guidelines,Central Steel Plant
1,Pellet Plant,Iron Scrap,tonnes,Q1,399043.61,2.849,tCO2/KL,1.136875e+06,91.64,90.20,IPCC 2006 Guidelines,Central Steel Plant
2,Pellet Plant,Petroleum Coke,tonnes,Q1,92899.41,1.390,tCO2/KL,1.291302e+05,75.73,87.63,IPCC 2006 Guidelines,Central Steel Plant
3,Pellet Plant,Bituminous Coal,tonnes,Q1,476400.55,3.084,tCO2/KL,1.469219e+06,81.49,93.63,IPCC 2006 Guidelines,Central Steel Plant
4,Pellet Plant,Calcined Petroleum Coke,tonnes,Q1,343775.39,2.036,tCO2/t,6.999267e+05,76.71,92.55,IPCC 2006 Guidelines,Central Steel Plant


In [3]:
# Filter and rename relevant columns
df_factors = df[[
    "Material",
    "Unit of Emission Factor",
    "Emission Factor",
    "Data Source for Emission Factor",
    "Location (Plant)",
    "Year/Timeline"
]].drop_duplicates()

# Rename columns to match API expectations
df_factors = df_factors.rename(columns={
    "Material": "material",
    "Unit of Emission Factor": "unit",
    "Emission Factor": "value",
    "Data Source for Emission Factor": "source",
    "Location (Plant)": "location",
    "Year/Timeline": "timeline"
})

In [4]:
df_factors.head()

,material,unit,value,source,location,timeline
0,Anthracite Coal,tCO2/t,1.044,IPCC 2006 Guidelines,Central Steel Plant,Q1
1,Iron Scrap,tCO2/KL,2.849,IPCC 2006 Guidelines,Central Steel Plant,Q1
2,Petroleum Coke,tCO2/KL,1.390,IPCC 2006 Guidelines,Central Steel Plant,Q1
3,Bituminous Coal,tCO2/KL,3.084,IPCC 2006 Guidelines,Central Steel Plant,Q1
4,Calcined Petroleum Coke,tCO2/t,2.036,IPCC 2006 Guidelines,Central Steel Plant,Q1


In [5]:
# Add versioning logic based on timeline
def infer_dates(timeline: str) -> tuple:
    year = 2024  # You can parameterize this
    if timeline == "Q1":
        return f"{year}-01-01", f"{year}-03-31"
    elif timeline == "Q2":
        return f"{year}-04-01", f"{year}-06-30"
    elif timeline == "Q3":
        return f"{year}-07-01", f"{year}-09-30"
    elif timeline == "Q4":
        return f"{year}-10-01", f"{year}-12-31"
    else:
        return f"{year}-01-01", f"{year}-12-31"

df_factors["start_date"], df_factors["end_date"] = zip(*df_factors["timeline"].map(infer_dates))

# Drop the timeline column
df_factors = df_factors.drop(columns=["timeline"])

In [6]:
df_factors.head()

,material,unit,value,source,location,start_date,end_date
0,Anthracite Coal,tCO2/t,1.044,IPCC 2006 Guidelines,Central Steel Plant,2024-01-01,2024-03-31
1,Iron Scrap,tCO2/KL,2.849,IPCC 2006 Guidelines,Central Steel Plant,2024-01-01,2024-03-31
2,Petroleum Coke,tCO2/KL,1.390,IPCC 2006 Guidelines,Central Steel Plant,2024-01-01,2024-03-31
3,Bituminous Coal,tCO2/KL,3.084,IPCC 2006 Guidelines,Central Steel Plant,2024-01-01,2024-03-31
4,Calcined Petroleum Coke,tCO2/t,2.036,IPCC 2006 Guidelines,Central Steel Plant,2024-01-01,2024-03-31


In [7]:
# Save to processed CSV
df_factors.to_csv("processed/emission_records.csv", index=False)

print("✅ Real processed emission_factors.csv created.")

✅ Real processed emission_factors.csv created.


In [8]:
df_2023 = df_factors.copy()
df_2023["start_date"] = pd.to_datetime("2023-01-01")
df_2023["end_date"] = pd.to_datetime("2023-12-31")

df_all = pd.concat([df_factors, df_2023], ignore_index=True)
df_all.to_csv("processed/emission_factors.csv", index=False)

In [11]:
# Load existing 2024 processed data
df_factors_2024 = pd.read_csv("processed/emission_factors.csv")

df_factors_2024["start_date"] = pd.to_datetime(df_factors_2024["start_date"], format='mixed', errors='coerce')
df_factors_2024["end_date"] = pd.to_datetime(df_factors_2024["end_date"], format='mixed', errors='coerce')

# Create 2023 copy by subtracting 1 year from each date
df_factors_2023 = df_factors_2024.copy()
df_factors_2023["start_date"] = df_factors_2023["start_date"] - pd.DateOffset(years=1)
df_factors_2023["end_date"] = df_factors_2023["end_date"] - pd.DateOffset(years=1)

# Combine 2023 + 2024
df_all = pd.concat([df_factors_2023, df_factors_2024], ignore_index=True)

# Save final merged data
df_all.to_csv("processed/emission_factors.csv", index=False)

print("✅ emission_factors.csv now includes both 2023 and 2024 data.")

✅ emission_factors.csv now includes both 2023 and 2024 data.


In [12]:
import pandas as pd
from datetime import datetime, date
import os

# Ensure the processed directory exists
os.makedirs("processed", exist_ok=True)

# Load Excel
df = pd.read_excel("data/emission_factors.xlsx")

# Filter and rename relevant columns
df_factors = df[[
    "Material",
    "Unit of Emission Factor",
    "Emission Factor",
    "Data Source for Emission Factor",
    "Location (Plant)",
    "Year/Timeline"
]].drop_duplicates()

# Rename columns to match API expectations
df_factors = df_factors.rename(columns={
    "Material": "material",
    "Unit of Emission Factor": "unit",
    "Emission Factor": "value",
    "Data Source for Emission Factor": "source",
    "Location (Plant)": "location",
    "Year/Timeline": "timeline"
})

# Add versioning logic based on timeline
def infer_dates(timeline: str, year: int = 2024) -> tuple:
    """Infer start and end dates from timeline string"""
    timeline_str = str(timeline).strip().upper()
    
    if timeline_str == "Q1":
        return f"{year}-01-01", f"{year}-03-31"
    elif timeline_str == "Q2":
        return f"{year}-04-01", f"{year}-06-30"
    elif timeline_str == "Q3":
        return f"{year}-07-01", f"{year}-09-30"
    elif timeline_str == "Q4":
        return f"{year}-10-01", f"{year}-12-31"
    else:
        # Default to full year
        return f"{year}-01-01", f"{year}-12-31"

# Process 2024 data
print("🔧 Processing 2024 data...")
df_2024 = df_factors.copy()
df_2024[["start_date", "end_date"]] = df_2024["timeline"].apply(
    lambda x: pd.Series(infer_dates(x, 2024))
)

# Process 2023 data
print("🔧 Processing 2023 data...")
df_2023 = df_factors.copy()
df_2023[["start_date", "end_date"]] = df_2023["timeline"].apply(
    lambda x: pd.Series(infer_dates(x, 2023))
)

# Combine 2023 and 2024 data
df_all = pd.concat([df_2023, df_2024], ignore_index=True)

# Drop the timeline column as it's no longer needed
df_all = df_all.drop(columns=["timeline"])

# Convert date columns to proper datetime format
df_all["start_date"] = pd.to_datetime(df_all["start_date"])
df_all["end_date"] = pd.to_datetime(df_all["end_date"])

# Save to processed CSV
output_path = "processed/emission_factors.csv"
df_all.to_csv(output_path, index=False)
print(f"✅ Saved {len(df_all)} emission factors to {output_path}")

# Display summary
print("\n📊 Summary:")
print(f"Total records: {len(df_all)}")
print(f"Date range: {df_all['start_date'].min()} to {df_all['end_date'].max()}")
print(f"Materials: {df_all['material'].nunique()}")
print(f"Locations: {df_all['location'].nunique()}")
print(f"Years covered: {sorted(df_all['start_date'].dt.year.unique())}")

# Show a sample of the data
print("\n🔍 Sample data:")
print(df_all.head())

# Create sample business metrics if it doesn't exist
business_metrics_path = "processed/business_metrics.csv"
if not os.path.exists(business_metrics_path):
    print(f"\n🔧 Creating sample business metrics...")
    
    # Generate sample business metrics for 2023 and 2024
    dates_2023 = pd.date_range(start="2023-01-01", end="2023-12-31", freq="M")
    dates_2024 = pd.date_range(start="2024-01-01", end="2024-12-31", freq="M")
    
    business_metrics = []
    
    # Add sample data for both years
    for date_val in list(dates_2023) + list(dates_2024):
        business_metrics.extend([
            {
                "date": date_val.strftime("%Y-%m-%d"),
                "metric_name": "Tons of Steel Produced",
                "value": 50000 + (date_val.month * 1000)  # Varying production
            },
            {
                "date": date_val.strftime("%Y-%m-%d"),
                "metric_name": "Number of Employees",
                "value": 1000 + (date_val.month * 10)  # Varying workforce
            }
        ])
    
    df_metrics = pd.DataFrame(business_metrics)
    df_metrics.to_csv(business_metrics_path, index=False)
    print(f"✅ Created sample business metrics: {len(df_metrics)} records")

print("\n🎉 Preprocessing complete! Your data is ready for use.")

🔧 Processing 2024 data...
🔧 Processing 2023 data...
✅ Saved 246 emission factors to processed/emission_factors.csv

📊 Summary:
Total records: 246
Date range: 2023-01-01 00:00:00 to 2024-06-30 00:00:00
Materials: 63
Locations: 1
Years covered: [np.int32(2023), np.int32(2024)]

🔍 Sample data:
                  material     unit  value                source  \
0          Anthracite Coal   tCO2/t  1.044  IPCC 2006 Guidelines   
1               Iron Scrap  tCO2/KL  2.849  IPCC 2006 Guidelines   
2           Petroleum Coke  tCO2/KL  1.390  IPCC 2006 Guidelines   
3          Bituminous Coal  tCO2/KL  3.084  IPCC 2006 Guidelines   
4  Calcined Petroleum Coke   tCO2/t  2.036  IPCC 2006 Guidelines   

              location start_date   end_date  
0  Central Steel Plant 2023-01-01 2023-03-31  
1  Central Steel Plant 2023-01-01 2023-03-31  
2  Central Steel Plant 2023-01-01 2023-03-31  
3  Central Steel Plant 2023-01-01 2023-03-31  
4  Central Steel Plant 2023-01-01 2023-03-31  

🎉 Preprocessing 

In [13]:
import pandas as pd
import os
from datetime import datetime, timedelta

def create_business_metrics():
    """Create business metrics file with sample data"""
    
    # Ensure processed directory exists
    os.makedirs("processed", exist_ok=True)
    
    # Create date range for both 2023 and 2024
    start_date_2023 = datetime(2023, 1, 1)
    end_date_2023 = datetime(2023, 12, 31)
    start_date_2024 = datetime(2024, 1, 1)
    end_date_2024 = datetime(2024, 12, 31)
    
    # Generate monthly data points
    business_metrics = []
    
    # Generate data for 2023
    current_date = start_date_2023
    while current_date <= end_date_2023:
        # Add monthly data for different metrics
        business_metrics.extend([
            {
                "date": current_date.strftime("%Y-%m-%d"),
                "metric_name": "Tons of Steel Produced",
                "value": 45000 + (current_date.month * 1500) + (current_date.month % 3 * 500)  # Monthly variation
            },
            {
                "date": current_date.strftime("%Y-%m-%d"),
                "metric_name": "Number of Employees",
                "value": 950 + (current_date.month * 8)
            },
            {
                "date": current_date.strftime("%Y-%m-%d"),
                "metric_name": "Production Hours",
                "value": 7200 + (current_date.month * 100)
            }
        ])
        
        # Move to next month
        if current_date.month == 12:
            current_date = current_date.replace(year=current_date.year + 1, month=1)
        else:
            current_date = current_date.replace(month=current_date.month + 1)
    
    # Generate data for 2024
    current_date = start_date_2024
    while current_date <= end_date_2024:
        # Add monthly data for different metrics with slight increase from 2023
        business_metrics.extend([
            {
                "date": current_date.strftime("%Y-%m-%d"),
                "metric_name": "Tons of Steel Produced",
                "value": 47000 + (current_date.month * 1600) + (current_date.month % 3 * 600)  # 2024 growth
            },
            {
                "date": current_date.strftime("%Y-%m-%d"),
                "metric_name": "Number of Employees",
                "value": 980 + (current_date.month * 9)
            },
            {
                "date": current_date.strftime("%Y-%m-%d"),
                "metric_name": "Production Hours",
                "value": 7400 + (current_date.month * 110)
            }
        ])
        
        # Move to next month
        if current_date.month == 12:
            current_date = current_date.replace(year=current_date.year + 1, month=1)
        else:
            current_date = current_date.replace(month=current_date.month + 1)
    
    # Create DataFrame
    df_metrics = pd.DataFrame(business_metrics)
    
    # Save to CSV
    output_path = "processed/business_metrics.csv"
    df_metrics.to_csv(output_path, index=False)
    
    print(f"✅ Created business metrics file: {output_path}")
    print(f"📊 Total records: {len(df_metrics)}")
    print(f"📊 Metrics: {df_metrics['metric_name'].unique()}")
    print(f"📊 Date range: {df_metrics['date'].min()} to {df_metrics['date'].max()}")
    
    # Show sample data
    print("\n🔍 Sample data:")
    print(df_metrics.head(10))
    
    return df_metrics

def verify_files():
    """Verify that all required files exist"""
    
    required_files = [
        "processed/emission_factors.csv",
        "processed/business_metrics.csv"
    ]
    
    print("🔍 Verifying required files...")
    
    for file_path in required_files:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            print(f"✅ {file_path} - {len(df)} records")
        else:
            print(f"❌ {file_path} - Missing!")
    
    # Check if emission_records.csv exists (created when you add records)
    records_path = "processed/emission_records.csv"
    if os.path.exists(records_path):
        df_records = pd.read_csv(records_path)
        print(f"✅ {records_path} - {len(df_records)} records")
    else:
        print(f"⚠️  {records_path} - Will be created when you add emission records")

if __name__ == "__main__":
    print("🚀 Creating business metrics for Carbon Emissions Platform...")
    print("=" * 60)
    
    # Create business metrics
    df_metrics = create_business_metrics()
    
    print("\n" + "=" * 60)
    print("🔍 Verifying all files...")
    
    # Verify files
    verify_files()
    
    print("\n" + "=" * 60)
    print("✅ Setup complete!")
    print("\n📋 Now you can test the intensity endpoint:")
    print("   GET http://127.0.0.1:8000/analytics/intensity")
    print("\n💡 Note: You need to have some emission records first.")
    print("   Add records using: POST http://127.0.0.1:8000/emission-records")

🚀 Creating business metrics for Carbon Emissions Platform...
✅ Created business metrics file: processed/business_metrics.csv
📊 Total records: 72
📊 Metrics: ['Tons of Steel Produced' 'Number of Employees' 'Production Hours']
📊 Date range: 2023-01-01 to 2024-12-01

🔍 Sample data:
         date             metric_name  value
0  2023-01-01  Tons of Steel Produced  47000
1  2023-01-01     Number of Employees    958
2  2023-01-01        Production Hours   7300
3  2023-02-01  Tons of Steel Produced  49000
4  2023-02-01     Number of Employees    966
5  2023-02-01        Production Hours   7400
6  2023-03-01  Tons of Steel Produced  49500
7  2023-03-01     Number of Employees    974
8  2023-03-01        Production Hours   7500
9  2023-04-01  Tons of Steel Produced  51500

🔍 Verifying all files...
🔍 Verifying required files...
✅ processed/emission_factors.csv - 246 records
✅ processed/business_metrics.csv - 72 records
✅ processed/emission_records.csv - 128 records

✅ Setup complete!

📋 Now you 

In [15]:
import requests
import json
import pandas as pd
import os
from datetime import datetime, date

def test_intensity_endpoint():
    """Test the intensity endpoint step by step"""
    
    base_url = "http://127.0.0.1:8000"
    
    print("🧪 Testing Intensity Endpoint...")
    print("=" * 50)
    
    # Step 1: Check if files exist
    print("1️⃣ Checking required files...")
    
    required_files = [
        "processed/emission_factors.csv",
        "processed/business_metrics.csv",
        "processed/emission_records.csv"
    ]
    
    for file_path in required_files:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            print(f"   ✅ {file_path} - {len(df)} records")
            
            # Show sample data
            if len(df) > 0:
                print(f"      📊 Columns: {list(df.columns)}")
                if "date" in df.columns:
                    print(f"      📅 Date range: {df['date'].min()} to {df['date'].max()}")
                elif "date_of_activity" in df.columns:
                    print(f"      📅 Date range: {df['date_of_activity'].min()} to {df['date_of_activity'].max()}")
        else:
            print(f"   ❌ {file_path} - Missing!")
    
    # Step 2: Add some emission records if none exist
    print("\n2️⃣ Ensuring emission records exist...")
    
    if not os.path.exists("processed/emission_records.csv") or len(pd.read_csv("processed/emission_records.csv")) == 0:
        print("   📝 Adding sample emission records...")
        
        sample_records = [
            {
                "material": "Petroleum Coke",
                "quantity": 100,
                "unit": "tCO2/KL",
                "date_of_activity": "2023-01-15",
                "location": "Central Steel Plant"
            },
            {
                "material": "Petroleum Coke",
                "quantity": 150,
                "unit": "tCO2/KL",
                "date_of_activity": "2023-02-15",
                "location": "Central Steel Plant"
            },
            {
                "material": "Natural Gas",
                "quantity": 200,
                "unit": "tCO2/m3",
                "date_of_activity": "2023-03-15",
                "location": "Central Steel Plant"
            },
            {
                "material": "Petroleum Coke",
                "quantity": 120,
                "unit": "tCO2/KL",
                "date_of_activity": "2024-01-15",
                "location": "Central Steel Plant"
            },
            {
                "material": "Petroleum Coke",
                "quantity": 180,
                "unit": "tCO2/KL",
                "date_of_activity": "2024-02-15",
                "location": "Central Steel Plant"
            }
        ]
        
        for record in sample_records:
            try:
                response = requests.post(f"{base_url}/emission-records", json=record)
                if response.status_code == 200:
                    print(f"   ✅ Added record for {record['date_of_activity']}")
                else:
                    print(f"   ❌ Failed to add record: {response.status_code} - {response.text}")
            except Exception as e:
                print(f"   ❌ Error adding record: {e}")
    
    # Step 3: Test the intensity endpoint
    print("\n3️⃣ Testing intensity endpoint...")
    
    try:
        response = requests.get(f"{base_url}/analytics/intensity")
        print(f"   📡 Response status: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"   ✅ Success! Result:")
            print(f"      {json.dumps(result, indent=2)}")
        else:
            print(f"   ❌ Error response:")
            print(f"      {response.text}")
            
    except Exception as e:
        print(f"   ❌ Request failed: {e}")
    
    # Step 4: Debug the function directly
    print("\n4️⃣ Testing function directly...")
    
    try:
        # Import and test the function directly
        from emissions_logic import get_emission_intensity
        
        result = get_emission_intensity()
        print(f"   ✅ Direct function call successful:")
        print(f"      {json.dumps(result, indent=2)}")
        
    except Exception as e:
        print(f"   ❌ Direct function call failed: {e}")
        import traceback
        traceback.print_exc()

def analyze_data_alignment():
    """Analyze if emission records and business metrics align properly"""
    
    print("\n🔍 Analyzing Data Alignment...")
    print("=" * 50)
    
    # Check emission records
    if os.path.exists("processed/emission_records.csv"):
        df_emissions = pd.read_csv("processed/emission_records.csv", parse_dates=["date_of_activity"])
        print(f"📊 Emission Records: {len(df_emissions)} records")
        
        if len(df_emissions) > 0:
            df_emissions["year_month"] = df_emissions["date_of_activity"].dt.to_period("M")
            emission_periods = df_emissions["year_month"].unique()
            print(f"   📅 Emission periods: {[str(p) for p in sorted(emission_periods)]}")
    
    # Check business metrics
    if os.path.exists("processed/business_metrics.csv"):
        df_metrics = pd.read_csv("processed/business_metrics.csv", parse_dates=["date"])
        print(f"📊 Business Metrics: {len(df_metrics)} records")
        
        if len(df_metrics) > 0:
            df_metrics["year_month"] = df_metrics["date"].dt.to_period("M")
            metric_periods = df_metrics["year_month"].unique()
            print(f"   📅 Metric periods: {[str(p) for p in sorted(metric_periods)]}")
            
            # Show available metrics
            metrics = df_metrics["metric_name"].unique()
            print(f"   📋 Available metrics: {metrics}")
            
            # Check for "Tons of Steel Produced"
            steel_metrics = df_metrics[df_metrics["metric_name"] == "Tons of Steel Produced"]
            if len(steel_metrics) > 0:
                steel_periods = steel_metrics["year_month"].unique()
                print(f"   🏭 Steel production periods: {[str(p) for p in sorted(steel_periods)]}")

if __name__ == "__main__":
    print("🚀 Debugging Intensity Endpoint...")
    print("=" * 60)
    
    # First analyze data alignment
    analyze_data_alignment()
    
    print("\n" + "=" * 60)
    
    # Then test the endpoint
    test_intensity_endpoint()
    
    print("\n" + "=" * 60)
    print("✅ Debugging complete!")

🚀 Debugging Intensity Endpoint...

🔍 Analyzing Data Alignment...
📊 Emission Records: 128 records
   📅 Emission periods: ['NaT', '2023-06', '2024-06']
📊 Business Metrics: 72 records
   📅 Metric periods: ['2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12']
   📋 Available metrics: ['Tons of Steel Produced' 'Number of Employees' 'Production Hours']
   🏭 Steel production periods: ['2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12']

🧪 Testing Intensity Endpoint...
1️⃣ Checking required files...
   ✅ processed/emission_factors.csv - 246 records
      📊 Columns: ['material', '

TypeError: '<=' not supported between instances of 'float' and 'str'